# Тема ВКР - плагин Visual Studio Code для рекомендации NuGet зависимостей на основе машинного обучения
#### В данной работе рассматривается разработка плагина, предназначенного для рекомендации NuGet зависимостейп о текстовому описанию разрабатываемого проекта.
#### Актуальность темы заключается в том, что во время разработки ПО приходится тратить значимую часть времени на поиск и установку необходимых библиотек в проект, что отвлекает от основной задачи и мешает сосредоточиться.
#### Целью работы является разработка плагина для Visual Studio Code, предназначенного для рекомендации NuGet зависимостей, чтобы избавить разработчика от рутинных действий по установке и поиску NuGet зависимостей.
#### Для достижения цели были поставлены следующие задачи:
*	Изучить основные алгоритмы обработки естественного языка с применением машинного обучения и методы разработки плагинов для Visual Studio Code
*	Составить методологию по реализации проекта
*	Осуществить сбор данных из открытых источников
*	Реализация модели машинного обучения и её тестирование
*	Создание плагина для Visual Studio Code с использованием разработанной модели машинного обучения


## Imports

In [ ]:
import csv
import requests
from bs4 import BeautifulSoup
import json
import time

In [ ]:
%pip install re
%pip install nltk
%pip install unicodedata
%pip install contractions
%pip install inflect
%pip install emoji

import re
import nltk
import emoji
import unicodedata
import contractions
import inflect
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')

In [ ]:
# Подготовка к частеречной разметке текста путём установки библиотеки Spacy, загрузки perceptron_tagger и модуля Spacy en
%pip install spacy
nltk.download('averaged_perceptron_tagger')
!python -m spacy download en_core_web_sm

import spacy

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 28.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


### базовые импорты

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import gc
%matplotlib inline

import sys
np.set_printoptions(suppress=True)
np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(precision=3)

DISPLAY_MAX_ROWS = 100 #20
pd.set_option('display.max_rows', DISPLAY_MAX_ROWS)
pd.set_option('display.max_column', 100) # None)
plt.style.use('seaborn-whitegrid')

import warnings
warnings.filterwarnings('ignore')

import re
import csv
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import CountVectorizer

<ipython-input-4-f5cc533de606>:20: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-whitegrid')


## Сбор данных

In [ ]:
import csv
import requests
from bs4 import BeautifulSoup
import json
import time

api_url = 'https://api.github.com/search/repositories'
headers = {'User-Agent': 'MyPyScrapper/1.0'}
repo_url = "https://api.github.com/repos/{}"
data_file = "repos_data.csv"

In [ ]:
# случай, если описание проекта не влазит полностью
def get_full_description(repo_name):
    # Send a GET request to the API endpoint
    response = requests.get(f'https://api.github.com/repos/{repo_name}')

    # Get the response content
    content = response.content.decode('utf-8')
    description = json.loads(content)['description']
    return description

def get_repo_data(url, headers, params):
    response = requests.get(url, headers=headers, params=params)

    # Check the response status code
    if response.status_code!= 200:
        print(f"Error {response.status_code}: {response.reason}")
        return []

    # Check the response content
    content = response.content.decode('utf-8')
    try:
        data = json.loads(content)
    except json.JSONDecodeError as e:
        print(f"JSON decode error: {e}")
        return []

    repos_collection = data['items'] # Extract the items array
    valid_repos = []
    for repo in repos_collection:
        repo_name = repo['full_name']
        repo_description = repo['description']

        if repo_description != "" and repo_description is not None:
            # описание не влазит полностью
            # поэтому дополнительно парсим страницу с репозиторием
            if len(repo_description) > 10 and repo_description.endswith('...'):
                repo_description = get_full_description(repo_name)
                print(f"делаем дополнительный парсинг к репозиторию {repo_name}")

            valid_repos.append([repo_name, repo_description])

    return valid_repos


def save_to_csv(data, file):
    fieldnames = ["Repository", "Description"]
    with open(file, "a", newline="", encoding="utf-8") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        if csvfile.tell() == 0:
            writer.writeheader()

        for row in data:
            writer.writerow({"Repository": row[0], "Description": row[1]})


In [ ]:
year = 2024
print(f"YEAR {year}")
for page in range(1, 5+1):
    params = {'q': f'created:{year}-01-01..{year}-12-31 language:C# is:public topics:<1', 'per_page': 200, 'page': page}
    print(f"Scrapping page {page}...")
    repos_data = get_repo_data(url=api_url, headers=headers, params=params)
    print(f"data: {repos_data}")

    if len(repos_data) > 0:
        save_to_csv(repos_data, data_file) # Сохранение элементов в csv формат

print()

YEAR 2024
Scrapping page 1...
data: [['xM4ddy/OFGB', 'GUI Tool To Removes Ads From Various Places Around Windows 11'], ['ExOK/Celeste64', 'A game made by the Celeste developers in a week(ish, closer to 2)'], ['FalconForceTeam/SOAPHound', 'SOAPHound is a custom-developed .NET data collector tool which can be used to enumerate Active Directory environments via the Active Directory Web Services (ADWS) protocol.'], ['unity3d-jp/Project_TCC', 'TCC stands for Tiny Character Controller. TCC is the best way to make your own game. This repository contains all packages and examples for TCC projects.'], ['AIDotNet/fast-wiki', '基于.NET8+React+LobeUI实现的企业级智能客服知识库'], ['vitoplantamura/HackerNewsRemovals', 'List of stories removed from the Hacker News Front Page, updated in real time.'], ['FireCubeStudios/StartStrikesBack', 'Fixed Windows 11 start menu by FireCube'], ['DeEpinGh0st/WindowsBaselineAssistant', 'Windows安全基线核查加固助手'], ['vercidium-patreon/meshing', 'Greedy meshing algorithm for voxel models']

## Предобработка текстового описания репозиториев

> C горем пополам собрал датасет из 8247 строк, cостоящий из 3-ёх типов запросов с 2019 по 2024 года

**`При применении методов машинного поиска для рекомендации NuGet зависимостей на основе текстовых описаний проектов, необходимо применять ту же самую предобработку к текстовому описанию, которое вводит пользователь, что и к данным в обучающем наборе, чтобы обеспечить согласованность данных и улучшить качество рекомендаций.`**

Далее последовательность действий следующая:
* Убрать специальные знаки, эмоджи
* Там куча разных языков миксованных с английским, нужно оставить чисто английский
* лемматизация/стемминг
* word embeddings/sentences embeddings



Перед векторизацией текста необходимо провести его предобработку, чтобы подготовить для обучения модели. Шаги, которые нужно выполнить:
1. Токенизация - процесс разбиения текста на небольшие фрагменты, называемые токенами. Токены могут быть отдельными словами, символами, фразами или другими единицами текста в зависимости от задачи и контекста.
2. Удаление стоп-слов - удаление слов, которые не несут значимой информации, таких как предлоги, артикли и т. д.
Нормализация - приведение текста к стандартному виду, например, замена всех букв на нижний регистр.
3. Лематизация - приведение слов к их словарной форме, например, глаголы в инфинитиве.
4. Стемминг - сокращение слова до его основы, удаляя окончания и суффиксы.
5. Расширение контекста - добавление слов вокруг целевого слова, чтобы увеличить контекст.
6. Фильтрация - удаление редких и шумных слов, которые могут ухудшить качество модели.
7. Разметка данных - разделение текста на тренировочную, тестовую и валидационную выборки.

>[Полезная статья с примером пайплайна для препроцессинга](https://habr.com/ru/articles/738176/) - это перевод, есть небольшие косяки, чекнуть комментарий к статье

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import gc
%matplotlib inline

import sys
np.set_printoptions(suppress=True)
np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(precision=3)

DISPLAY_MAX_ROWS = 100 #20
pd.set_option('display.max_rows', DISPLAY_MAX_ROWS)
pd.set_option('display.max_column', 100) # None)
plt.style.use('seaborn-whitegrid')

import warnings
warnings.filterwarnings('ignore')

<ipython-input-1-5bcc61d61cd7>:20: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-whitegrid')


In [2]:
# Load the CSV dataset
data = pd.read_csv("/content/repos_data.csv")
data.drop_duplicates()
data.dropna()
data

,Repository,Description
0,microsoft/PowerToys,Windows system utilities to maximize productivity
1,2dust/v2rayN,"A GUI client for Windows, support Xray core an..."
2,files-community/Files,Building the best file manager for Windows
3,huiyadanli/RevokeMsgPatcher,:trollface: A hex editor for WeChat/QQ/TIM - P...
4,netchx/netch,A simple proxy client
...,...,...
8242,fgilde/AuralizeBlazor,AuralizeBlazor is a wrapper component for audi...
8243,AcK77/TTGames-Explorer-Rebirth,TTGames modding tool
8244,mhdbouk/multitenantdemo,Source code for the YouTube video `Is Your .NE...
8245,CrazyZhang666/MarneTools,战地1马恩工具箱


In [3]:
print(data.isna().sum())
print()
print(data.isnull().sum())

Repository     0
Description    0
dtype: int64

Repository     0
Description    0
dtype: int64


### Очистка мусора, конвертация эмоджи

> Есть описания состоящие из непонятных мне иероглифов, после обработки описание становится пустым - с этим нужно что-то делать (либо терять данные - удалять, либо запариться и переводить на английский, но могут появиться неточности перевода и появится шум...)

In [4]:
%pip install re
%pip install nltk
%pip install unicodedata
%pip install contractions
%pip install inflect
%pip install emoji

import re
import nltk
import emoji
import unicodedata
import contractions
import inflect
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')

ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re
ERROR: Could not find a version that satisfies the requirement unicodedata (from versions: none)
ERROR: No matching distribution found for unicodedata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.8/433.8 kB 5.9 MB/s eta 0:00:00


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

Разбиение на пробелы может привести к тому, что то, что следует рассматривать как один токен, может быть разбито на два, как в случае с некоторыми именами собственными (например, Сан-Франциско или Нью-Йорк) или заимствованными иностранными фразами (например, laissez faire).

**Токенизация** также может удалить знаки препинания, упрощая путь к правильной сегментации слов, но также потенциально вызывая другие сложности. В случае точек, которые следуют за аббревиатурой (например, dr.), точка после этой аббревиатуры должна рассматриваться как часть того же токена и не удаляться.

Процесс токенизации может быть особенно проблематичным при работе с биомедицинскими текстами, которые содержат много дефисов, скобок и других знаков препинания.


In [5]:
# Функция для очистки текста
def clean_text(input_text):

    # HTML-теги: первый шаг - удалить из входного текста все HTML-теги
    clean_text = re.sub('<[^<]+?>', '', input_text)

    # URL и ссылки: далее - удаляем из текста все URL и ссылки
    clean_text = re.sub(r'http\S+', '', clean_text)

    # Эмоджи и эмотиконы: удаляем их нафиг, это кринж какой-то бессмысленный, только шума добавят
    clean_text = remove_emojis(clean_text)

    # Приводим все входные данные к нижнему регистру
    clean_text = clean_text.lower()

    # Убираем все пробелы
    # Так как все данные теперь представлены словами - удалим пробелы
    clean_text = re.sub('\s+', ' ', clean_text)

    # Преобразование символов с диакритическими знаками к ASCII-символам: используем функцию normalize из модуля unicodedata и преобразуем символы с диакритическими знаками к ASCII-символам
    clean_text = unicodedata.normalize('NFKD', clean_text).encode('ascii', 'ignore').decode('utf-8', 'ignore')

    # Разворачиваем сокращения: текст часто содержит конструкции вроде "don't" или "won't", поэтому развернём подобные сокращения
    clean_text = contractions.fix(clean_text)

    # Специальные случаи для языков программирования
    clean_text = re.sub(r'c\#', 'csharp', clean_text)
    clean_text = re.sub(r'c\+\+', 'cpp', clean_text)

    # Убираем специальные символы: избавляемся от всего, что не является "словами"
    clean_text = re.sub('[^a-zA-Z0-9\s]', '', clean_text)

    # Записываем числа прописью: 100 превращается в "сто" (для компьютера)
    temp = inflect.engine()
    words = []
    for word in clean_text.split():
        if word.isdigit():
            words.append(temp.number_to_words(word))
        else:
            words.append(word)
    clean_text = ' '.join(words)

    # Стоп-слова: удаление стоп-слов - это стандартная практика очистки текстов
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(clean_text)
    tokens = [token for token in tokens if token not in stop_words]
    clean_text = ' '.join(tokens)

    # Add full-stop to end of sentences
    clean_text = re.sub('([a-z])\.([A-Z])', r'\1. \2', clean_text)

    # Знаки препинания: далее - удаляем из текста все знаки препинания
    clean_text = re.sub(r'[^\w\s]', '', clean_text)

    # И наконец - возвращаем очищенный текст
    return clean_text

# Функция для преобразования эмоджи в слова
def emojis_to_txt(text):

    # Модуль emoji: преобразование эмоджи в их словесные описания
    clean_text = emoji.demojize(text, delimiters=(" ", " "))

    # Редактирование текста путём замены ":" и" _", а так же - путём добавления пробела между отдельными словами
    clean_text = clean_text.replace(":", "").replace("_", " ")

    return clean_text

def remove_emojis(text):
    emoji_pattern = re.compile(
        "["
        "\U00010000-\U0010FFFF"  # Unicode characters beyond the Basic Multilingual Plane
        "\u200d"  # Zero-width joiner
        "\u2640-\u2642"  # Male and female signs
        "\u2600-\u26FF"  # Miscellaneous symbols
        "\u2700-\u27BF"  # Dingbats
        "]+",
        flags=re.UNICODE
    )
    return emoji_pattern.sub(r'', text)

In [6]:
test_with_emojis = "😊 This is a sample 😊text witgh an emoji 😊 [blablabla] (^🧩) :trollface:"
text_without_emojis = remove_emojis(test_with_emojis)
print(text_without_emojis)

 This is a sample text witgh an emoji  [blablabla] (^) :trollface:


### Лемматизация и Частеречная разметка текста

In [ ]:
# Подготовка к частеречной разметке текста путём установки библиотеки Spacy, загрузки perceptron_tagger и модуля Spacy en
%pip install spacy
nltk.download('averaged_perceptron_tagger')
!python -m spacy download en_core_web_sm

import spacy

nlp = spacy.load('en_core_web_sm')
lemmatize_exceptions = ['ios', 'mmorts']

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
def lemmatize_and_postag(input_text):

    doc = nlp(input_text)
    lemmatize_output = []

    # Iterate over each token in the document
    for token in doc:
        if token.text in lemmatize_exceptions:
            lemmatize_output.append(token.text + '_' + token.pos_)
        else:
            # Append the token lemma and its POS tag to the tagged_output list
            lemmatize_output.append(token.lemma_ + '_' + token.pos_)

    # Join the tagged_output list into a single string
    lemmatize_output_str = ' '.join(lemmatize_output)

    return lemmatize_output_str

In [ ]:
def preprocesse(input_text):
    clear_text = clean_text(input_text)
    #print("\nОчистка мусора, конвертация эмоджи:\n", clear_text)

    tagged_output = lemmatize_and_postag(clear_text)
    #print("\nЧастеречная разметка и лемматизация:\n", tagged_output)

    return tagged_output

## Собираем все вместе и предобрабатываем

In [ ]:
data.isna().sum()

Repository     0
Description    0
dtype: int64

In [ ]:
data.Description = data.Description.apply(clean_text)

In [ ]:
data.to_csv("clean_data.csv", index=False, columns=["Repository", "Description"])

In [ ]:
data.isna().sum()

Repository     0
Description    0
dtype: int64

In [ ]:
data

,Repository,Description
0,microsoft/PowerToys,windows system utilities maximize productivity
1,2dust/v2rayN,gui client windows support xray core v2fly cor...
2,files-community/Files,building best file manager windows
3,huiyadanli/RevokeMsgPatcher,trollface hex editor wechatqqtim pcqqtim
4,netchx/netch,simple proxy client
...,...,...
8242,fgilde/AuralizeBlazor,auralizeblazor wrapper component audiomotionan...
8243,AcK77/TTGames-Explorer-Rebirth,ttgames modding tool
8244,mhdbouk/multitenantdemo,source code youtube video net app multitenant ...
8245,CrazyZhang666/MarneTools,one


Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.
Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.


In [ ]:
clean_data = pd.read_csv("/content/clean_data.csv")
clean_data

,Repository,Description
0,microsoft/PowerToys,window_NOUN system_NOUN utility_NOUN maximize_...
1,2dust/v2rayN,gui_PROPN client_PROPN window_VERB support_PRO...
2,files-community/Files,build_VERB good_ADJ file_NOUN manager_NOUN win...
3,huiyadanli/RevokeMsgPatcher,trollface_VERB hex_PROPN editor_NOUN wechatqqt...
4,netchx/netch,simple_ADJ proxy_ADJ client_NOUN
...,...,...
8242,fgilde/AuralizeBlazor,auralizeblazor_PROPN wrapper_PROPN component_N...
8243,AcK77/TTGames-Explorer-Rebirth,ttgame_NOUN modding_NOUN tool_NOUN
8244,mhdbouk/multitenantdemo,source_NOUN code_PROPN youtube_NOUN video_NOUN...
8245,CrazyZhang666/MarneTools,one_NUM


In [ ]:
clean_data.isna().sum()

Repository       0
Description    109
dtype: int64

In [ ]:
clean_data = clean_data.dropna()
clean_data.shape

(8138, 2)

In [ ]:
clean_data.to_csv("preprocessed_txtdata.csv", index=False, columns=["Repository", "Description"])

## Векторизация

In [ ]:
%pip install gensim
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import FastText
import gensim
from sklearn.neighbors import NearestNeighbors

In [ ]:
data = pd.read_csv("/content/preprocessed_txtdata.csv")
data

,Repository,Description
0,microsoft/PowerToys,window_NOUN system_NOUN utility_NOUN maximize_...
1,2dust/v2rayN,gui_PROPN client_PROPN window_VERB support_PRO...
2,files-community/Files,build_VERB good_ADJ file_NOUN manager_NOUN win...
3,huiyadanli/RevokeMsgPatcher,trollface_VERB hex_PROPN editor_NOUN wechatqqt...
4,netchx/netch,simple_ADJ proxy_ADJ client_NOUN
...,...,...
8133,fgilde/AuralizeBlazor,auralizeblazor_PROPN wrapper_PROPN component_N...
8134,AcK77/TTGames-Explorer-Rebirth,ttgame_NOUN modding_NOUN tool_NOUN
8135,mhdbouk/multitenantdemo,source_NOUN code_PROPN youtube_NOUN video_NOUN...
8136,CrazyZhang666/MarneTools,one_NUM


In [ ]:
data['Description'] = data['Description'].apply(lambda x: [word.split('_')[0] for word in x.split()]) # сразу токенизируем и убираем POS теги
data.Description.sample(5)

1846    [unity, csharp, api, connection, stablediffusi...
2130        [open, source, csharp, tool, make, net, easy]
5771    [sharpgpoabuse, net, application, write, cshar...
2599    [automatically, import, texture, material, amb...
1479    [patch, version, unity3d, unityhub, windows, m...
Name: Description, dtype: object



> model - embeeding model (FastText)

> knn - KNN kd-tree model of documents' vectors



In [ ]:
# Preprocess the text data
def preprocess_query(query_text):
    query_text = preprocesse(query_text)
    return query_text.split()

# get the vector representation of a query
def get_query_vector(query, model):
    words = preprocess_query(query)
    query_vector = model.wv.get_sentence_vector(words)
    return query_vector

# get the vector representation of a repository
def get_repository_vector(description, model):
    words = description
    repo_vector = model.wv.get_sentence_vector(words)
    return repo_vector

# function to retrieve relevant repositories for a given query
def retrieve_relevant_repositories(query, knn, model, data):
    query_vector = get_query_vector(query, model)
    distances, indices = knn.kneighbors([query_vector])
    return data.Repository.iloc[indices[0]]

#### FastText + kNN(kd-tree)

##### Простой пример

In [ ]:
import numpy as np
import pandas as pd
import gensim
from sklearn.model_selection import GroupKFold, GridSearchCV
from sklearn.base import BaseEstimator, ClassifierMixin

from sklearn.metrics import f1_score, make_scorer
from sklearn.model_selection import GridSearchCV

# Define the custom scorer function
f1_scorer = make_scorer(f1_score, average='weighted')

# Assume you have a pandas dataframe 'data' with columns 'Description' and 'Repository'
data = pd.DataFrame({
    'Description': [
        "The quick brown fox jumps over the lazy dog",
        "The lazy dog sleeps on the couch",
        "The quick fox eats the slow rabbit",
        "The slow rabbit hops away",
        "The brown fox has a bushy tail"
    ],
    'Repository': [1, 2, 3, 4, 5]
})

# Preprocess the data by tokenizing the descriptions
data['Description'] = data['Description'].apply(lambda x: x.split())

# Define the hyperparameter space for the FastText model
param_grid = {
    'epochs': [5, 10],
    'vector_size': [5, 10, 15],
    'window': [3, 5, 7, 10],
    'min_count': [1, 5, 10, 20],
    'workers': [2, 4, 8]
}


class FastTextEstimator(BaseEstimator, ClassifierMixin):
    def __init__(self, epochs=5, vector_size=20, window=3, min_count=1, workers=2):
        self.epochs = epochs
        self.vector_size = vector_size
        self.window = window
        self.min_count = min_count
        self.workers = workers

    def fit(self, X, y=None):
        self.model = gensim.models.FastText(X, sg=0, hs=1, epochs=self.epochs, vector_size=self.vector_size, window=self.window, min_count=self.min_count, workers=self.workers)
        return self

    def get_params(self, deep=False):
        return {
            'epochs': self.epochs,
            'vector_size': self.vector_size,
            'window': self.window,
            'min_count': self.min_count,
            'workers': self.workers
        }

    def set_params(self, **params):
        for key, value in params.items():
            setattr(self, key, value)
        return self


# Perform grid search to find the best hyperparameters using GroupKFold
gkf = GroupKFold(n_splits=3)
grid_search = GridSearchCV(
    estimator=FastTextEstimator(),
    param_grid=param_grid,
    cv=gkf.split(data.Description.values, groups=data.Repository.values),
    scoring=f1_scorer,
    n_jobs=-1,
    verbose=2)

grid_search.fit(data.Description.values)

# Print the best hyperparameters and the corresponding F1 score
print("Best hyperparameters:", grid_search.best_params_)
print("Best score:", grid_search.best_score_)

# Train the FastText model with the best hyperparameters
model = FastTextEstimator(**grid_search.best_params_)
model.fit(data.Description.values)

Fitting 3 folds for each of 288 candidates, totalling 864 fits
Best hyperparameters: {'epochs': 5, 'min_count': 1, 'vector_size': 5, 'window': 3, 'workers': 2}
Best score: nan


In [ ]:
context = ['the', 'quick', 'brown', '__unknown__']

# Predict the most similar word to the unknown word in the context
predicted_word = grid_search.best_estimator_.model.wv.most_similar(positive=context[:-1], negative=[context[-1]])[:3]

print(predicted_word)

[('sleeps', 0.7607722878456116), ('has', 0.7237035632133484), ('rabbit', 0.6555864810943604)]


##### Реальный пример

In [ ]:
import numpy as np
import pandas as pd
import gensim
from sklearn.model_selection import GroupKFold
from sklearn.base import BaseEstimator, ClassifierMixin
from gensim.similarities import Similarity

In [ ]:
# Define the FastText model
model = FastText(data.Description.values,
                 sg=0,
                 hs=1,
                 negative=0,
                 alpha=0.025,
                 vector_size=60,
                 window=3,
                 min_count=3,
                 workers=5,
                 epochs=7)

# Train the model
model.train(data.Description.values, total_examples=model.corpus_count, epochs=50)

In [ ]:
# Save the model to a file
model.save("fasttext_model")

In [ ]:
sim_score = model.wv.most_similar('webs')[:5]
sim_score if sim_score[0][1] > 0.55 else "ничего особенного"

[('web3', 0.804424524307251),
 ('web', 0.7525070309638977),
 ('website', 0.6947178840637207),
 ('websocket', 0.6846100687980652),
 ('webui', 0.6841009855270386)]



> пример автодополнения по контексту



In [ ]:
context_test1 = ['jwt' 'au', '__unknown__']
context_test2 = ['clean', 'archit', '__unknown__']

# Predict the most similar word to the unknown word in the context
predicted_word1 = model.wv.most_similar(positive=context_test1[:-1], negative=[context_test1[-1]])[:5]
print(predicted_word1)

predicted_word2 = model.wv.most_similar(positive=context_test2[:-1], negative=[context_test2[-1]])[:5]
print(predicted_word2)

[('jwt', 0.7600553035736084), ('tokens', 0.4985504746437073), ('angular', 0.43316516280174255), ('webhook', 0.425396203994751), ('materiais', 0.42355403304100037)]
[('architecture', 0.6033294200897217), ('eventdriven', 0.5702003240585327), ('practical', 0.5392873287200928), ('architectural', 0.5324025750160217), ('principle', 0.5294102430343628)]




> применение в машином поиске



**Векторизация каждого репозитория**

In [ ]:
# Calculate the vector representation of each repository description
repository_vectors = data.Description.apply(lambda description: get_repository_vector(description, model))

In [ ]:
repository_vectors.sample(5).values

array([array([ 0.022,  0.022, -0.068, -0.001, -0.005,  0.053, -0.024,  0.042,
               0.007, -0.012, -0.037,  0.031,  0.075,  0.013,  0.036,  0.022,
              -0.036, -0.048,  0.021, -0.011,  0.008, -0.048,  0.008, -0.035,
              -0.046,  0.019,  0.028, -0.025,  0.001, -0.035, -0.052,  0.02 ,
              -0.026, -0.077,  0.04 ,  0.025,  0.024,  0.012, -0.017,  0.037,
               0.062, -0.062,  0.09 , -0.012, -0.008, -0.04 ,  0.059,  0.04 ,
              -0.053,  0.023, -0.042,  0.092,  0.014,  0.003, -0.03 , -0.022,
               0.007,  0.07 ,  0.041,  0.03 ], dtype=float32)                ,
       array([-0.011, -0.04 ,  0.   ,  0.054, -0.062,  0.003, -0.03 , -0.042,
               0.056,  0.031, -0.034,  0.059,  0.07 , -0.006,  0.024, -0.142,
              -0.07 ,  0.056,  0.021,  0.099,  0.008, -0.115,  0.029,  0.017,
              -0.039,  0.016,  0.017,  0.032,  0.044, -0.138,  0.074, -0.012,
              -0.003, -0.019, -0.016, -0.052,  0.02 ,  0.051, -

**обучение векторной модели машинного поиска**

In [ ]:
# Convert the Series to a list of vectors and then to a NumPy array
repository_vectors_array = np.array(repository_vectors.tolist())

# Train the KNN model
knn = NearestNeighbors(n_neighbors=5, algorithm='kd_tree')
knn.fit(repository_vectors_array)

NearestNeighbors(algorithm='kd_tree')

In [ ]:
import joblib

# Save the KNN model
joblib.dump(knn, 'knn_model.joblib')

['knn_model.joblib']

## Загрузка моделей и использование

In [ ]:
import gc
gc.collect()

719

In [ ]:
fasttext_model = FastText.load("fasttext_model")

In [ ]:
knn = joblib.load('knn_model.joblib')

In [ ]:
# Retrieve relevant repositories for the query "realtime chat app"
query = "jwt auth"
relevant_repositories = retrieve_relevant_repositories(query, knn, fasttext_model, data)

print(relevant_repositories)

5394                              TeshaneCrawford/jwtAuth
6948        cornflourblue/dotnet-5-jwt-refresh-tokens-api
6818        cornflourblue/dotnet-6-jwt-authentication-api
1597    NikiforovAll/keycloak-authorization-services-d...
6391    cornflourblue/aspnet-core-3-jwt-refresh-tokens...
Name: Repository, dtype: object
